# gestion des secrets

[source](https://savelev.medium.com/best-practice-how-to-store-secrets-and-settings-in-python-project-e3ee45b3094c)

In [146]:
import os
import json
def read_secrets() -> dict:
    filename = os.path.join('secrets.json')
    try:
        with open(filename, mode='r') as f:
            return json.loads(f.read())
    except FileNotFoundError:
        return {}
secrets = read_secrets()

# test accès stripe

In [2]:
import stripe
stripe.api_key = secrets['stripe.api_key_test']

# list customers
customers = stripe.Customer.list()

# print the first customer's email
print(customers.data[0].email)

# retrieve specific Customer
# customer = stripe.Customer.retrieve("cus_123456789")

# print that customer's email
# print(customer.email)

cha.le@hotmail.fr


# exeuction rapport

In [3]:
import datetime
import time

In [25]:
from tqdm import tqdm
for i in tqdm(range(10000000)):
    pass

100%|██████████| 10000000/10000000 [00:01<00:00, 6958596.86it/s]


## gestion date début/fin rapport

[lien vers doc de stripe](https://stripe.com/docs/api/reporting/report_run/object)

[différents type de rapport stripe](https://stripe.com/docs/reports/report-types)

In [151]:
import stripe
import requests
import pandas as pd
import io
from woocommerce import API
from datetime import datetime

stripe.api_key = secrets['stripe.api_key']

def convert_to_timestamp(date):
	return int(time.mktime(datetime.strptime(date, "%d/%m/%Y").timetuple()))

def convert_to_isoformat(date_timestamp):
	return datetime.fromtimestamp(date_timestamp).isoformat()

def get_stripe_data(start_date, end_date, PAUSE=10):
  report = stripe.reporting.ReportRun.create(
    report_type="payout_reconciliation.itemized.5",
    parameters={
      "interval_start": start_date,
      "interval_end": end_date,
      "columns": ['automatic_payout_id', 
      'automatic_payout_effective_at',
      'balance_transaction_id',
      'created', 
      'available_on',
      'currency',
      'gross',
      'fee',
      'net',
      'reporting_category',
      'source_id',
      'description',
      'customer_id',
      'customer_email',
      'customer_name',
      'customer_description',
      'invoice_id',
      'order_id',
      'payment_method_type'
      ]
    },
  )


  timeout = time.time() + 60*20   # 20 minutes from now
  i=0

  id = report['id']

  while time.time() < timeout:
      report = stripe.reporting.ReportRun.retrieve(id)
      if report['status'] == 'succeeded':
          print(f"Execution {i+1} +{PAUSE*i}sec : {report['status']}")
          r = requests.get(report['result']['url'], auth=(stripe.api_key,""))
          urlData = r.content
          df = pd.read_csv(io.StringIO(urlData.decode('utf-8')))
          print('stripe_data_ok')
          break
      else :
          print(f"Execution {i+1} +{PAUSE*i}sec : {report['status']}")
      i +=1
      time.sleep(PAUSE)
  return df


def get_woocommerce_data(start_date, end_date):
	#init api
  wcapi = API(
    url=secrets['woocommerce.url'], # Your store URL
    consumer_key=secrets['woocommerce.key'], # Your consumer key
    consumer_secret=secrets['woocommerce.secret'], # Your consumer secret
    wp_api=True, # Enable the WP REST API integration
    version="wc/v3" # WooCommerce WP REST API version
  )
  nb_commandes = wcapi.get("orders")
  nb_commandes = nb_commandes.headers['X-WP-Total']
  commandes = wcapi.get(f"orders?per_page=30&after={start_date_isoformat}&before={end_date_isoformat}")
  df_commandes = pd.DataFrame(commandes.json())
  print('woocommerce_data_ok')
  return df_commandes   

In [152]:
PAUSE = 60
start_date = "01/10/2022"
end_date = "31/10/2022"
start_date_timestamp = convert_to_timestamp(start_date)
end_date_timestamp = convert_to_timestamp(end_date)
start_date_isoformat = convert_to_isoformat(start_date_timestamp)
end_date_isoformat = convert_to_isoformat(end_date_timestamp)

df_stripe = get_stripe_data(start_date_timestamp, end_date_timestamp)
df_woocommerce = get_woocommerce_data(start_date_timestamp, end_date_timestamp)


Execution 1 +0sec : pending
Execution 2 +10sec : pending
Execution 3 +20sec : pending
Execution 4 +30sec : succeeded
stripe_data_ok
woocommerce_data_ok
